In [1]:
#importing the libraries
import numpy as np
import pandas as pd

In [37]:
#reading the data
df = pd.read_csv("FoDS-Assignment-2.csv")

# shuffle the DataFrame rows
df = df.sample(frac = 1)

# df.isnull().sum()

In [3]:
df['sqft_living'] = df.fillna(value = df['sqft_living'].mean())
df['floors'] = df.fillna(value = df['floors'].mean())
df['sqft_above'] = df.fillna(value = df['sqft_above'].mean())


df.isnull().sum()

bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
sqft_living15    0
sqft_lot15       0
price            0
dtype: int64

In [4]:
#Feature Scaling
#normalising the data values
df = (df - df.min()) / (df.max() - df.min())

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,sqft_living15,sqft_lot15,price
1148,0.142857,0.090909,0.142857,0.006342,0.142857,0.0,0.0,0.666667,0.250,0.142857,0.000000,0.295276,0.018625,0.080403
837,0.428571,0.227273,0.428571,0.176651,0.428571,0.0,0.0,0.666667,0.250,0.428571,0.000000,0.165354,0.053825,0.055128
375,0.285714,0.227273,0.285714,0.005750,0.285714,0.0,0.0,0.333333,0.250,0.285714,0.000000,0.234252,0.018354,0.023886
1110,0.428571,0.318182,0.428571,0.006971,0.428571,0.0,0.0,0.666667,0.500,0.428571,0.000000,0.348425,0.021668,0.131868
344,0.428571,0.090909,0.428571,0.006909,0.428571,0.0,0.0,1.000000,0.250,0.428571,0.000000,0.110236,0.028029,0.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,0.285714,0.227273,0.285714,0.003052,0.285714,0.0,0.0,0.666667,0.250,0.285714,0.266667,0.108268,0.008803,0.074176
243,0.428571,0.272727,0.428571,0.004031,0.428571,0.0,0.0,1.000000,0.250,0.428571,0.293333,0.151575,0.015090,0.038187
492,0.285714,0.318182,0.285714,0.028607,0.285714,0.0,0.0,0.333333,0.375,0.285714,0.116667,0.283465,0.054300,0.068681
760,0.285714,0.272727,0.285714,0.000308,0.285714,0.0,0.0,0.333333,0.250,0.285714,0.100000,0.127953,0.001328,0.023443


In [5]:
#splitting the dataset into training data and testing data
splitData = int(0.7*len(X))
train_X, test_X, train_y, test_y = X[:splitData], X[splitData:], y[:splitData], y[splitData:]

In [6]:
WeightV = np.zeros(14)
for n in range(14):
    WeightV[n] = np.random.randn()

In [7]:
#finding the weights based on the training data
def fit(X, Y, iters, learning_rate, c, d):
    for itr in range(iters):
        sumItrError = 0    
        for z in range(len(X)):        # each row in input data
            dataP_error = 0            # calculating error in each data point
            
            for m in range(13):
                dataP_error += (c[m] + d[m]) * WeightV[m+1] * X[z][m]    # summation of (w1*x1 + w2*x2 + w3*x3 + w4*x4 ...)
                
            dataP_error += WeightV[0]
            dataP_error -= Y[z]      # (w0 + w1*x1 + w2*x1^2 + w3*x1*x2 + w4*x2^2 ...) - yn

            # for each parameter(w0, w1, w2,...)    
            for m in range(14):                   
                if(m == 0):
                    WeightV[m] -= (learning_rate/len(X)) * dataP_error
                else:
                    WeightV[m] -= (learning_rate/len(X)) * dataP_error * X[z][m-1]   # calculating w0, w1, w2,... for each iteration
        
            dataP_error = (dataP_error**2)
            sumItrError += dataP_error/(2*len(Y))
            
        sumItrError = (sumItrError)**0.5
#         if (itr == iters-1): 
#             print("Minimum training error is ", sumItrError)
    return sumItrError

In [8]:
def predict(X, Y, c, d):
    sumItrError = 0
    for z in range(len(X)):        # each row in input data
        dataP_error = 0            # calculating error in each data point
            
        for m in range(13):
            dataP_error += (c[m] + d[m]) * WeightV[m+1] * X[z][m]    # summation of (w1*x1 + w2*x2 + w3*x3 + w4*x4 ...)
        dataP_error += WeightV[0]
        
        dataP_error -= Y[z]      # (w0 + w1*x1 + w2*x1^2 + w3*x1*x2 + w4*x2^2 ...) - yn
        dataP_error = (dataP_error**2)
        sumItrError += dataP_error/(2*len(X))
            
    sumItrError = (sumItrError)**0.5
    return sumItrError    


In [9]:
c = np.ones(13)
d = np.zeros(13)

#fitting the training data
trainingE = fit(train_X, train_y, 1000, 1, c, d)
print("Minimum training error is ", trainingE)

#making predictions on test data
testingE = predict(test_X, test_y, c, d)
print("Minimum testing error is ", testingE)
 

Minimum training error is  0.03855091181683963
Minimum testing error is  0.036988309159919144
